In [1]:
import pandas as pd
import numpy as np
import joblib
import sys, os
from sklearn.preprocessing import KBinsDiscretizer, Normalizer

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
progs = []
for file in os.listdir('D:/AlgoMLData/Progressions/'):
    print(file)
    progs.extend(joblib.load('D:/AlgoMLData/Progressions/{}'.format(file)))

AUDNZD_progs.pkl
AUDUSD_progs.pkl
CADJPY_progs.pkl
CHFJPY_progs.pkl
EURCHF_progs.pkl
EURGBP_progs.pkl
EURJPY_progs.pkl
EURUSD_progs.pkl
GBPUSD_progs.pkl
NZDUSD_progs.pkl
USDJPY_progs.pkl


### Candle Size Priors

In [3]:
## Get the max drawdown
risk_multiplier = 2.5
n_bins = 8

In [4]:
success = []
failure = []
css = []

for prog in progs:
    cs = prog.CandleSize.values[0]
    css.append(cs)
    max_drawdown = cs * risk_multiplier + cs
    x = prog[prog.RDD > max_drawdown]
    if x.shape[0] != 0:
        failure.append(cs)
    else:
        success.append(cs)
len(failure) + len(success), len(progs)

(80541, 80541)

In [5]:
cs_encoder = KBinsDiscretizer(n_bins=n_bins, encode='ordinal', strategy='uniform')
cs_encoder = cs_encoder.fit(np.reshape(css, (-1, 1)))
css_bins = cs_encoder.transform(np.reshape(css, (-1, 1))).reshape(-1, )

In [6]:
success = [cs_encoder.transform([[c]])[0][0] for c in success]
failure = [cs_encoder.transform([[c]])[0][0] for c in failure]
prob_success = len(success) / len(css)
prob_failure = len(failure) / len(css)

In [7]:
len(failure), len(success)

(16628, 63913)

In [8]:
## Raw Probabilities
_, counts = np.unique(css_bins, return_counts=True)
counts = np.append(counts, [0]*min(0, n_bins - counts.shape[0]))
raw_cs_probs = counts / counts.sum()
raw_cs_probs

array([9.45530848e-01, 5.24825865e-02, 1.65133286e-03, 2.11072621e-04,
       8.69122559e-05, 2.48320731e-05, 1.24160366e-05])

In [9]:
_, counts = np.unique(success, return_counts=True)
counts = np.append(counts, [0]*max(0, n_bins - counts.shape[0]))
raw_success_probs = counts / counts.sum()
raw_success_probs

array([9.46145542e-01, 5.16796270e-02, 1.81496722e-03, 2.03401499e-04,
       1.09523884e-04, 3.12925383e-05, 1.56462691e-05, 0.00000000e+00])

In [10]:
_, counts = np.unique(failure, return_counts=True)
counts = np.append(counts, [0]*max(0, n_bins - counts.shape[0]))
raw_failure_probs = counts / counts.sum()
raw_failure_probs

array([9.43168150e-01, 5.55689199e-02, 1.02237190e-03, 2.40558095e-04,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00])

In [11]:
def candle_success(cs):
    candle_bin = int(cs_encoder.transform([[cs]])[0][0])
    p_s_b = raw_success_probs[candle_bin] * prob_success
    p_s_b /= raw_success_probs[candle_bin] * prob_success + raw_failure_probs[candle_bin] * prob_failure
    return p_s_b

### Drawdown conditionals

In [12]:
drawdowns = {}
dd_encoders = {}
candle_drawdowns = {
    i : [] for i in range(100)
}
n_bins = 124
max_risk = 2.5

In [13]:
for file in os.listdir("D:/AlgoMLData/Drawdowns/"):
    ticker = file.split('_')[0]
    with open("D:/AlgoMLData/Drawdowns/" + file, 'rb') as file:
        drawdowns[ticker] = joblib.load(file)

In [14]:
for ticker in drawdowns:
    for candle in drawdowns[ticker]:
        candle_drawdowns[candle].extend(drawdowns[ticker][candle])

In [15]:
for i in candle_drawdowns:
    kbd = KBinsDiscretizer(n_bins=n_bins, encode='ordinal', strategy='uniform')
    kbd = kbd.fit(np.reshape(candle_drawdowns[i], (-1, 1)))
    dd_encoders[i] = kbd

In [16]:
success = {
    i : [] for i in range(100)
}
failure = {
    i : [] for i in range(100)
}
for prog in progs:
    cs = prog.CandleSize.values[0]
    mdd = cs * max_risk
    x = prog[prog.RDD > mdd]
    if x.shape[0] == 0:
        for i, rdd in enumerate(prog.RDD[1:]):
            success[i].append(dd_encoders[i].transform([[rdd]])[0][0])
    else:
        for i, rdd in enumerate(prog[prog.RDD <= mdd].RDD[1:]):
            failure[i].append(dd_encoders[i].transform([[rdd]])[0][0])

In [17]:
dd_success = []
dd_failure = []

for i in success:
    _, counts = np.unique(success[i], return_counts=True)
    counts = np.append(counts, [0]*max(0, n_bins - counts.shape[0]))
    counts = counts / counts.sum()
    dd_success.append(counts.tolist())
    
for i in failure:
    _, counts = np.unique(failure[i], return_counts=True)
    counts = np.append(counts, [0]*max(0, n_bins - counts.shape[0]))
    counts = counts / counts.sum()
    dd_failure.append(counts.tolist())
    
dd_success = np.array(dd_success).T
dd_failure = np.array(dd_failure).T

C:\Users\Zach\Anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in true_divide
  del sys.path[0]


P(S | D1) = (P(D1|S) * P(S)) / (P(D1|S) * P(S) + P(D1|F) * P(F))

### Trade Example

### Function to calculate it

In [33]:
ctr = 0

In [42]:

for trade in progs[ctr:ctr+1]:
    
    cs = trade.CandleSize.values[0]
    css = candle_success(cs)
    bvf, bvs = [], []
    
    for i, rdd in enumerate(trade.RDD[1:]):
        rdd_bin = int(dd_encoders[0].transform([[rdd]])[0][0])
        bvs.append(dd_success[rdd_bin, i])
        bvf.append(dd_failure[rdd_bin, i])
        p_s_d = np.cumprod(bvs)[-1] * css
        p_s_d /= np.cumprod(bvs)[-1] * css + np.cumproduct(bvf)[-1] * (1 - css)
        print('Candle Size', cs, 'Candle Success', css, 'Drawdown', rdd - cs, 'Probability', p_s_d)
ctr+=1

Candle Size 293.49999999999096 Candle Success 0.7814052519517388 Drawdown 56.000000000011596 Probability 0.7317179030655693
Candle Size 293.49999999999096 Candle Success 0.7814052519517388 Drawdown 25.000000000008356 Probability 0.31092749899098554
Candle Size 293.49999999999096 Candle Success 0.7814052519517388 Drawdown -0.49999999998107114 Probability 0.10248853392490634
Candle Size 293.49999999999096 Candle Success 0.7814052519517388 Drawdown -250.49999999999793 Probability 0.09372349904359227
Candle Size 293.49999999999096 Candle Success 0.7814052519517388 Drawdown -293.49999999999096 Probability 0.39738429061225183
